In [3]:
import numpy as np
import random
import pyomo.environ as pyo
from abc import ABC

BIG_M = 10000

# narrativa
"""_summary_
coupon non agli attivi
"""


'_summary_\ncoupon non agli attivi\n'

In [2]:
environ_params = {
    'reward_per_active' : 15,
    'reward_per_semi_active' : 10,
    
    'action_email_cost' : 10,
    'action_coupon_cost' : 10,
    'action_resurrection_cost' : 10,
    'action_special_cost' : 10,

    'epoch_new_active' : 10,
    'epoch_new_semi_active' : 5,
    'epoch_active_to_semi_active': .02,
    'epoch_semi_active_to_non_active': .05,
    
    'email_semi_active_to_active' : (.1, .005),
    'email_non_active_to_active' : (.05, .005),
    'email_non_active_to_semi_active' : (.1, .02),
    'email_2_active_to_non_active': (.05, .005),
    'email_2_active_to_semi_active' : (.15, .02),
    'email_2_semi_active_to_non_active' : (.1 , .02),
    
    'coupon_semi_active_to_active' : (.5, .02),
    'coupon_non_active_to_active' : (.2, .005),
    'coupon_active_to_non_active' : (.02, .001),
    
    'resurrection_non_active_to_active' : (.4, .02),
    'resurrection_non_active_to_semi_active' : (.1, .015),
    'resurrection_active_to_semi_active' : (.05, .005),
    'resurrection_semi_active_to_non_active' : (.05, .005),

    'special_semi_active_to_active' : (.1, .005),
    'special_non_active_to_semi_active' : (.05, .005),

    'new_active' : 10,
    'new_semi_active' : 5,

    'step_active_to_semi_active' : (.02, .005),
    'step_semi_active_to_non_active' : (.05, .005),
}




def customer_transition(source_number, params):
    loc, scale = params
    # Sample from Normal random variable
    transation = np.random.normal(loc, scale, 1)[0]
    # Make sample a proportion
    transation = max( min(1,transation), 0)
    # Find the number of transiting customers
    transation *= source_number
    # Cast into int
    transation = int(transation)

    return transation






In [3]:
customer_transition(1000, (.10, .02))


133

In [4]:

class Environment():

    def __init__(self, active_customers, semi_active_customers, non_active_customers):
        self._active_customers = active_customers
        self._semi_active_customers = semi_active_customers
        self._non_active_customers = non_active_customers

        self._active_customers_final = 0
        self._semi_active_customers_final = 0
        self._non_active_customers_final = 0

        #self._active_customers_next_transitions = 0
        #self._semi_active_customers_next_transitions = 0
        #self._non_active_customers_next_transitions = 0


    def __str__(self):
        return f"Active: \t {self._active_customers}\nSemi act: \t {self._semi_active_customers}\nNon act \t {self._non_active_customers}\n"


    # Good transitions
        
    def _semi_active_to_active(self, param):
        number = customer_transition(self._semi_active_customers, param)
        self._semi_active_customers_final -= number
        self._active_customers_final += number

    def _non_active_to_active(self, param):
        number = customer_transition(self._non_active_customers, param)
        self._non_active_customers_final -= number
        self._active_customers_final += number

    def _non_active_to_semi_active(self, param):
        number = customer_transition(self._non_active_customers, param)
        self._non_active_customers_final -= number
        self._semi_active_customers_final += number


    # Bad transitions
        
    def _active_to_semi_active(self, param):
        number = customer_transition(self._active_customers, param)
        self._active_customers_final -= number
        self._semi_active_customers_final += number
        
    def _active_to_non_active(self, param):
        number = customer_transition(self._active_customers, param)
        self._active_customers_final -= number
        self._non_active_customers_final += number

    def _semi_active_to_non_active(self, param):
        number = customer_transition(self._semi_active_customers, param)
        self._semi_active_customers_final -= number
        self._non_active_customers_final += number


    # Actions    
        
    def _step_organic(self):
        self._active_to_semi_active(environ_params.get('step_active_to_semi_active'))
        self._semi_active_to_non_active(environ_params.get('step_semi_active_to_non_active'))

    def _action_email(self):
        self._semi_active_to_active(environ_params.get('email_semi_active_to_active'))
        self._non_active_to_active(environ_params.get('email_non_active_to_active'))
        self._non_active_to_semi_active(environ_params.get('email_non_active_to_semi_active'))

        # Should be 2 step ahead
        self._active_to_non_active(environ_params.get('email_2_active_to_non_active'))
        self._active_to_semi_active(environ_params.get('email_2_active_to_semi_active'))
        self._semi_active_to_non_active(environ_params.get('email_2_semi_active_to_non_active'))


    def _action_coupon(self):
        self._semi_active_to_active(environ_params.get('coupon_semi_active_to_active'))
        self._non_active_to_active(environ_params.get('coupon_non_active_to_active'))
        self._active_to_non_active(environ_params.get('coupon_active_to_non_active'))

    def _action_resurrection(self):
        self._non_active_to_active(environ_params.get('resurrection_non_active_to_active'))
        self._non_active_to_semi_active(environ_params.get('resurrection_non_active_to_semi_active'))
        self._active_to_semi_active(environ_params.get('resurrection_active_to_semi_active'))
        self._semi_active_to_non_active(environ_params.get('resurrection_semi_active_to_non_active'))

    def _action_special(self):
        self._semi_active_to_active(environ_params.get('special_semi_active_to_active'))
        self._non_active_to_semi_active(environ_params.get('special_non_active_to_semi_active'))


    # State transition
        
    def step(self, action : str = None):

        # New organic customers
        self._active_customers += environ_params.get('new_active')
        self._semi_active_customers += environ_params.get('new_semi_active')

        # Setup the customers classes after the action
        self._active_customers_final = self._active_customers
        self._semi_active_customers_final = self._semi_active_customers
        self._non_active_customers_final = self._non_active_customers

        # Do step organic transitions
        self._step_organic()

        # Calculate reward
        reward = self._active_customers * environ_params.get('reward_per_active') + \
                 self._semi_active_customers * environ_params.get('reward_per_semi_active')

        # Do action and update the costs
        if action == 'email':
            self._action_email()
            reward -= (self._active_customers + self._semi_active_customers + self._non_active_customers) * environ_params.get('action_email_cost')
        elif action == 'coupon':
            self._action_coupon()
            reward -= (self._semi_active_customers + self._non_active_customers) * environ_params.get('action_coupon_cost')
        elif action == 'resurrection':
            self._action_resurrection()
            reward -= (self._non_active_customers) * environ_params.get('action_resurrection_cost')
        elif action == 'special':
            self._action_special()
            reward -= (self._active_customers + self._semi_active_customers + self._non_active_customers) * environ_params.get('action_special_cost')

        # Update customer classes
        self._active_customers = max(0, self._active_customers_final)
        self._semi_active_customers = max(0, self._semi_active_customers_final)
        self._non_active_customers = max(0, self._non_active_customers_final)

        return reward

    def make_step(self, action : str = None):
        print(f"Reward: \t {self.step(action)}\n - \n")


    # For VFA

    def export_status(self):
        return (self._active_customers, self._semi_active_customers, self._non_active_customers)
        





In [5]:
my_environment = Environment(active_customers=1500, semi_active_customers=1200, non_active_customers=2300)

In [6]:
print(my_environment)

Active: 	 1500
Semi act: 	 1200
Non act 	 2300



In [7]:
my_environment.make_step()
print(my_environment)

Reward: 	 34700
 - 

Active: 	 1483
Semi act: 	 1163
Non act 	 2369



In [8]:
my_environment.make_step()
print(my_environment)

Reward: 	 34075
 - 

Active: 	 1471
Semi act: 	 1135
Non act 	 2424



In [9]:
my_environment.make_step('email')
print(my_environment)

Reward: 	 -16835
 - 

Active: 	 1451
Semi act: 	 1260
Non act 	 2334



In [10]:
my_environment.make_step()
print(my_environment)

Reward: 	 34565
 - 

Active: 	 1425
Semi act: 	 1243
Non act 	 2392



In [11]:
my_environment.make_step('special')
print(my_environment)

Reward: 	 -16745
 - 

Active: 	 1539
Semi act: 	 1186
Non act 	 2350



In [12]:
my_environment.make_step()
print(my_environment)

Reward: 	 35145
 - 

Active: 	 1518
Semi act: 	 1160
Non act 	 2412



## Value Function Approximation


In [13]:
class VFA_Agent():

    def __init__(self, actions, gamma, alpha, epsilon):

        self.gamma = gamma
        self.alpha = alpha
        self.epsilon = epsilon # Exploration-exploitation balance parameter

        self._actions = actions
        self._actions_dim = len(actions)

        self._state = (0, 0, 0)

        self._tile_coding_coef = 100

        self._state_active_dim = 100
        self._state_semi_active_dim = 100
        self._state_non_active_dim = 100

        self._q_function = np.zeros([self._state_active_dim, self._state_semi_active_dim, self._state_non_active_dim, self._actions_dim])


    def _tile_coding(self, export):
        (num_active, num_semi_active, num_non_active) = export
        return (int(num_active/self._tile_coding_coef), 
                       int(num_semi_active/self._tile_coding_coef), 
                       int(num_non_active/self._tile_coding_coef))


    def _set_state(self, export):
        self._state = self._tile_coding(export)

        
    def _select_action(self):
        """
        Select best action considering exploration-exploitation balance
        """
        if np.random.rand() < self.epsilon:
            return random.randint(0, self._actions_dim - 1)
        else:
            q_values = [self._q_function[self._state + (a,)] for a in range(self._actions_dim)]
            # Break the ties!
            best_actions = np.where(q_values == np.max(q_values))[0]
            return np.random.choice(best_actions)
        
    def _update_q_function(self, action, reward, next_state):
        """
        Q-Learning core: this fuction calculates the update of the q function
        """
        
        # get the current value of the q function for the given state and action 
        current_q = self._q_function[self._state + (action,)]
        
        # get the q function value for the prescribed action for the next state
        max_next_q = np.max(self._q_function[next_state])

        # calculate the temporal difference  (Q - learning)
        td_error = reward + self.gamma * max_next_q - current_q

        # update the q function (Q - learning)
        self._q_function[self._state + (action,)] += self.alpha * td_error

    def train(self, environment: Environment, num_episodes):
        """
        Run simulation to learn the q function
        """
        
        # Training loop
        for episode in range(num_episodes):
            
            # Set the status from the environment status
            self._set_state(environment.export_status())

            # Choose an action
            action = self._select_action()

            # Step the environment, save the reward
            reward = environment.step(self._actions[action])

            # Save the new state
            new_state = self._tile_coding(environment.export_status())

            # Update the q function 
            self._update_q_function(action, reward, new_state)


    def get_value_function(self, export):
        state = self._tile_coding(export)
        q_values = self._q_function[state, :]
        return np.mean(q_values)


    

In [14]:
actions = [None, 'email', 'coupon', 'resurrection', 'special']
agent = VFA_Agent(actions, gamma=0.9, alpha=0.1, epsilon=0.1)

In [15]:
for _ in range(100):
    my_environment = Environment(active_customers=1500, semi_active_customers=1200, non_active_customers=2300)
    agent.train(my_environment, 100)

In [16]:
print(my_environment)

Active: 	 3922
Semi act: 	 1690
Non act 	 888



In [17]:
agent.get_value_function(my_environment.export_status())

10.46420716139705

In [18]:
agent._q_function

array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        ...,

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         

## Direct LookAhead


In [19]:
class DLA_Agent():

    def __init__(self, actions):

        #self.gamma = gamma
        #self.alpha = alpha
        #self.epsilon = epsilon # Exploration-exploitation balance parameter

        self._actions = actions
        self._actions_dim = len(actions)

        self._state = (0, 0, 0)

        self._tile_coding_coef = 10

        self._state_active_dim = 1000
        self._state_semi_active_dim = 1000
        self._state_non_active_dim = 1000

        self._q_function = np.zeros([self._actions_dim])


    def _tile_coding(self, export):
        (num_active, num_semi_active, num_non_active) = export
        return (int(num_active/self._tile_coding_coef), 
                       int(num_semi_active/self._tile_coding_coef), 
                       int(num_non_active/self._tile_coding_coef))


    def _set_state(self, export):
        self._state = self._tile_coding(export)

        
    def _select_action(self):
        """
        Select best action using euristic policy
        """
        return random.randint(0, self._actions_dim - 1)


    def train(self, initial_environment, num_episodes, len_episodes):
        """
        Run simulation to learn the q function
        """

        # Loop on the action to be trained
        for action_train in range(self._actions_dim): 

            rewards = []
        
            # Training loop
            for _ in range(num_episodes):
                
                environment = Environment(active_customers=initial_environment.get('active_customers'), 
                                          semi_active_customers=initial_environment.get('semi_active_customers'), 
                                          non_active_customers=initial_environment.get('non_active_customers'))

                for step in range(len_episodes):
                
                    # Set the status from the environment status
                    self._set_state(environment.export_status())

                    # Choose an action, if this is the first step, the use the action in training
                    if step == 0:
                        action = action_train
                    else:
                        action = self._select_action()

                    # Step the environment, save the reward
                    rewards.append( environment.step(self._actions[action]) )


            self._q_function[action_train] = np.mean(rewards)


    def get_value_function(self):
        return self._q_function
        


    

In [20]:
actions = [None, 'email', 'coupon', 'resurrection', 'special']

agent = DLA_Agent(actions)

initial_environment = {'active_customers': 1500,
                       'semi_active_customers': 1200,
                       'non_active_customers': 2300,}



agent.train(initial_environment=initial_environment, num_episodes=50, len_episodes=10)

In [21]:
print(agent.get_value_function())


[22331.7  15017.77 22511.63 27844.55 16454.63]


## Cost Function Approximation


In [13]:
num_step = 3
scenarios_by_step = 6


all_scenarios = list(range(1, scenarios_by_step**num_step+1))


high_wind_probability = .6
low_wind_probability = .4

low_demand_probability = .3
medium_demand_probability = .4
high_demand_probability = .3


demands = {1: [17000, 12000, 10000],
           2: [12000, 10000, 80000],
           3: [18000, 15000, 90000]}



data = {}

data.update({'S' : all_scenarios})
data.update({'T' : list(range(1, num_step+1))})


non_ant_sets = {}
for t in range(num_step):
    elements = scenarios_by_step**(num_step-t)
    groups = scenarios_by_step**(t)
    non_ant_sets.update({t+1 : [ all_scenarios[i:i+elements] for i in range(0, groups) ]})

data.update({'NonAnt' : non_ant_sets})


data.update({'cost_activate_nuc' : {None: 30000}})
data.update({'power_nuc' : {None: 15000}})

data.update({'cost_activate_gas' : {None: 50}})
data.update({'cost_kw_gas' : {None: 3}})
data.update({'max_power_gas' : {None: 5000}})

data.update({'cost_kw_wind' : {None: 2}})

data.update({'power_wind_high' : {None: 7000}})
data.update({'power_wind_low' : {None: 2000}})
data.update({'cost_kw_ext' : {None: 10}})

wind_high_param = {}
for t in range(num_step):
    elements = int(scenarios_by_step**(num_step-t)/2)
    my_list = [1]*elements + [0]*elements
    my_list = my_list*(scenarios_by_step**(t))

    for s, element in enumerate(my_list):
        wind_high_param.update({(t+1, s+1):element})

wind_low_param = {}
for t in range(num_step):
    elements = int(scenarios_by_step**(num_step-t)/2)
    my_list = [0]*elements + [1]*elements
    my_list = my_list*(scenarios_by_step**(t))

    for s, element in enumerate(my_list):
        wind_low_param.update({(t+1, s+1):element})

data.update({'wind_high' : wind_high_param}) 
data.update({'wind_low' : wind_low_param}) 


prob_param = {}

single_scenario_probabilities = [high_wind_probability * high_demand_probability, 
                                high_wind_probability * medium_demand_probability,
                                high_wind_probability * low_demand_probability,
                                low_wind_probability  * high_demand_probability,
                                low_wind_probability  * medium_demand_probability,
                                low_wind_probability  * low_demand_probability]

all_probabilities = [A * B * C \
                     for A in single_scenario_probabilities
                     for B in single_scenario_probabilities
                     for C in single_scenario_probabilities]

for s, element in enumerate(all_probabilities):
    prob_param.update({s+1 : element})

data.update({'prob' : prob_param})



demand_param = {}
for t in range(num_step):

    current_demand_profile = demands[t+1]

    current_demand_profile = current_demand_profile*2

    multiply = ((scenarios_by_step)**(num_step-t-1))

    current_demand_profile = [item for item in current_demand_profile for _ in range(multiply)]

    groups = scenarios_by_step**t

    current_demand_profile = current_demand_profile*groups

    for s, element in enumerate(current_demand_profile):
        demand_param.update({(t+1,s+1) : element})

data.update({'demand' : demand_param}) # !!!!

data = {None : data}


In [88]:


    # Objective function
def ObjRule(cfa):
        return sum( 

            (
                cfa.nuc_act[s] * cfa.cost_activate_nuc + \
                sum(cfa.gas_act[t,s] * cfa.cost_activate_gas for t in cfa.T) + \
                sum(cfa.gas_qnt[t,s] * cfa.cost_kw_gas for t in cfa.T) + \
                sum(
                    (cfa.wind_high[t,s] * cfa.power_wind_high + cfa.wind_low[t,s] * cfa.power_wind_low )
                    * cfa.cost_kw_wind for t in cfa.T ) +\
                
                sum(cfa.ext_qnt[t,s] * cfa.cost_kw_ext for t in cfa.T)
            ) * cfa.prob[s]
        
            
            for s in cfa.S)


    # Constraints

def maxPowerGasRule(cfa, t, s):
        return cfa.gas_qnt[t,s] <= cfa.max_power_gas
    
def activationGas(cfa, t, s):
        return cfa.gas_qnt[t,s] <= cfa.gas_act[t,s] * BIG_M

def satisfyDemand(cfa, t, s):
        return cfa.demand[t,s] <= cfa.power_nuc * cfa.nuc_act[s] + \
            cfa.gas_qnt[t,s] + \
            cfa.ext_qnt[t,s] + \
            cfa.wind_high[t,s] * cfa.power_wind_high + cfa.wind_low[t,s] * cfa.power_wind_low
            
            
def NonAnt_nuc_act(cfa, s):
        return cfa.nuc_act[1] == cfa.nuc_act[s]

def NonAnt_gas_act(cfa, t, s):
        return cfa.gas_act[t,1] == cfa.gas_act[t,s[1]]



#cfa.NonAnt_gas_act = pyo.Constraint(cfa.T, cfa.NonAnt, rule = NonAnt_gas_act)
#cfa.NonAnt_gas_qnt = pyo.Constraint(cfa.T, cfa.NonAnt, rule = NonAnt_gas_qnt)
#cfa.NonAnt_ext_qnt = pyo.Constraint(cfa.T, cfa.NonAnt, rule = NonAnt_ext_qnt)
            
            




cfa = pyo.AbstractModel() # pyomo abstract model

        ### MODEL

        # Sets
        #cfa.S = pyo.RangeSet(1, cfa.n_scenarios) # Set of the scenarios
        #cfa.T = pyo.RangeSet(1, cfa.n_steps)

cfa.S = pyo.Set(initialize = list(range(1, scenarios_by_step**num_step+1)))
cfa.S.construct()
cfa.T = pyo.Set(initialize = list(range(1, num_step+1)))
cfa.T.construct()
cfa.NonAnt = pyo.Set(cfa.T, initialize = non_ant_sets)
cfa.NonAnt.construct()


        # Parameters

cfa.cost_activate_nuc = pyo.Param() # Activate nuclear in one step 
cfa.power_nuc = pyo.Param()    # Power provided from nuclear (constant)
        
cfa.cost_activate_gas = pyo.Param() # Activate gas in one step
cfa.cost_kw_gas = pyo.Param()  # Cost per kw from gas
cfa.max_power_gas = pyo.Param()# Max power from gas

cfa.cost_kw_wind = pyo.Param() # Cost per kw from wind
cfa.power_wind_high = pyo.Param()
cfa.power_wind_low = pyo.Param()

cfa.wind_high = pyo.Param(cfa.T, cfa.S) # 1 on wind high scenarions, 0 on wind low ones
cfa.wind_low = pyo.Param(cfa.T, cfa.S) # 1 on wind low scenarions, 0 on wind high ones

#cfa.prob_wind_high = pyo.Param()  # forse non serve
#cfa.prob_wind_low = pyo.Param()  # forse non serve
cfa.prob = pyo.Param(cfa.S)

cfa.cost_kw_ext = pyo.Param()  # Soft demand satisfaciton constraints

cfa.demand = pyo.Param(cfa.T, cfa.S)


        # Variables
cfa.nuc_act = pyo.Var(cfa.S, within=pyo.Binary)
cfa.gas_act = pyo.Var(cfa.T, cfa.S, within=pyo.Binary)
cfa.gas_qnt = pyo.Var(cfa.T, cfa.S, within=pyo.NonNegativeReals)
cfa.ext_qnt = pyo.Var(cfa.T, cfa.S, within=pyo.NonNegativeReals)


        # Objective function
cfa.obj = pyo.Objective(rule=ObjRule, sense=pyo.minimize)
        

        # Constraints
cfa.maxPowerGas = pyo.Constraint(cfa.T, cfa.S, rule=maxPowerGasRule)
cfa.activationGas = pyo.Constraint(cfa.T, cfa.S, rule=activationGas)
cfa.satisfyDemand = pyo.Constraint(cfa.T, cfa.S, rule=satisfyDemand)

cfa.NonAnt_nuc_act = pyo.Constraint(cfa.NonAnt[1].data()[0], rule = NonAnt_nuc_act)
cfa.NonAnt_gas_act = pyo.Constraint(cfa.T, cfa.NonAnt, rule = NonAnt_gas_act)
cfa.NonAnt_gas_qnt = pyo.Constraint(cfa.T, cfa.NonAnt, rule = NonAnt_gas_qnt)
cfa.NonAnt_ext_qnt = pyo.Constraint(cfa.T, cfa.NonAnt, rule = NonAnt_ext_qnt)



#cfa.NonAnt_nuc_act = pyo.Constraint(cfa.NonAnt[1], rule = NonAnt_nuc_act)






"""
def create_instance(self,data):
        data = {None : data}
        self._instance = dla.create_instance(data)

def solve(self): #!!!!!
        # Solver configuration
        self._solver = pyo.SolverFactory('appsi_highs')
                #solver.options['Threads'] = config['SOLVER']['SolverThreads']
                #solver.options['TimeLimit'] = config['SOLVER']['SolverTimeLimit']
                #solver.options['MIPGap'] = config['SOLVER']['SolverMIPGap']

        # Solver launching
        solver_result = self._solver.solve(self._instance, tee=False)
                
        # Saving data of the solution
        self._instance.solutions.store_to(solver_result)

        return self._instance # questa adrebbe processata dentro una schedula prima di procedere
            
            
"""


TypeError: Cannot apply a Set operator to an indexed Set component (NonAnt)

In [80]:
data.get('cost_activate_nuc')

In [81]:
instance = cfa.create_instance(data)

In [82]:
solver = pyo.SolverFactory('appsi_highs')
                #solver.options['Threads'] = config['SOLVER']['SolverThreads']
                #solver.options['TimeLimit'] = config['SOLVER']['SolverTimeLimit']
                #solver.options['MIPGap'] = config['SOLVER']['SolverMIPGap']

solver_result = solver.solve(instance, tee=False)
                
instance.solutions.store_to(solver_result)


In [83]:
non_ant_sets

{1: [[1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   125,
   126,
   127,
   128,
   129,
   130,
   131,
   132,
   133,
   134,
   135,
   136,
   137,
   138,
 

In [87]:
for i in range(216):
    if instance.nuc_act[i+1].value != instance.nuc_act[1].value:
        print('hello!')

In [42]:
instance.gas_act[1,1].value

-0.0

In [43]:
instance.gas_qnt[1,1].value

0.0

In [44]:
instance.ext_qnt[1,1].value

0.0

In [47]:
instance.S.data()

(1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [77]:
for i in instance.NonAnt[1].data()[0]:


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
